# Introduction

According to the World Health Organization (WHO), Stroke is the second cause of death worldwide. It is a high morbidity, disability and mortality disease and whose victims may suddenly experience paralysis, impaired speed, or loss of vision due to the interruption of blood flow(ischemia) caused by thrombosis and embolism. 
Although  the pathogenesis of Stroke is not quite clear, it is known to be closely related to abnormal metabolic indicators, of which 90% can be controlled. This is why predicting stroke can be determining for the individuals to be treated earlier and avoid disabling and mortal consequences.

# Objective

The purpose of this research is to apply Data Science and Machine Learning techniques to create an accurate model based on Stroke dataset to preduct stroke outcome from knowing some individuals characteristics and provide useful information of whether a person is susceptible to stroke and help medical staff to take required actions to lower the existing risk and avoid its consequences.

# Methodology

The methodology to be used for this study is the CRISP-DM methodology, one of the most widely used methodology for  Data Analysis projects. It consists in 6 principal stages listed as follows:
- Business Understanding
- Data Understanding
- Data Preparation
- Modeling
- Evaluation
- Deployment

# 1. Business Understanding 

This first stage of the CRISP-DM methodology aims for the data analyst to understand the business and the problem to be solved in order to take the required actions while analyzing the available data.
In this case, we are talking about the medicine field. Which is one of the areas where Data Science is getting a considering importance because of its efficiency and accuracy. As we are helping to detect a mortal disease, it is very important that the dataset is managed the best possible and evaluate the cost of getting wrong predictions (false positives and false negatives) in order to decide what to do in these scenarios.

# 2. Data Understanding

The Stroke Prediction Dataset used for this investigation was retrieved Kaggle and can be obtained from the follwing link:
https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

Now let´s see what kind of information it contains and identify how it can be used for the previously described objective.

In [ ]:
# Import required libraries to import and manipulate dataset
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

## Reading Data

In [ ]:
# Read the data in "Stroke" dataset
ds= pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
ds

## Feature Description

We can see that the dataset contains information about 5110 individuals, and the features included are:
- id: Individual identifier
- gender: Individual gender
- age: Age of the individual
- hypertension: Indicates whether a person has hyperthension or not
- heat disease: Indicates wheter the person suffers from heart disease
- ever married: Indicates if the person has been married
- work type: Individual work type
- residence type: Type of residence the person has
- Average glucose level: Average glucose level of the individual
- bmi: Individual body mass index 
- Smoking status: Indicates the smoking status
- Stroke: stroke outcome of individuals within the dataset

## Missing Data and Dropping 'id'

First of all, we can get rid of information that is not required such as the 'id', which is not more that an individual identifier and provides no useful information in this case.

In [ ]:
ds.drop(['id'],axis=1, inplace=True)
ds.head()

Now let´s check if we have any missing information within the dataset.

In [ ]:
ds.isnull().sum()

We are missing BMI for 201 individuals. We can fill this missing data by using the mean.

In [ ]:
# Calculating BMI mean to fill missing values
bmimean = ds['bmi'].mean()
print(bmimean)

In [ ]:
# Check for missing data again
ds = ds.fillna(bmimean)
ds.isnull().sum()

Now that we have no more missing data, let´s see some description about it to identify what kinf of information we are dealing with and some useful statistics about it, as well as the type of data for each of the features described

In [ ]:
ds.dtypes

In [ ]:
# Description of data
ds.describe()

From the description, we can identify the values ranges for numerical features, which are: age, bmi and glucose level:
- age: 0.08 - 82 years
- average glucose level: 55.12 - 271.74
- BMI: 10.3 - 97.6

It might be better to round the ages we are more used to use it that way

In [ ]:
# Rounding Age
ds['age'] = ds['age'].apply(lambda x: round(x))

We also have both cathegorical and binary features, so we might take a look on their possible values to understand better the information they provide.

In [ ]:
print(ds['gender'].unique())
print(ds['hypertension'].unique())
print(ds['heart_disease'].unique())
print(ds['ever_married'].unique())
print(ds['work_type'].unique())
print(ds['Residence_type'].unique())
print(ds['smoking_status'].unique())
print(ds['stroke'].unique())

## Gender classification

We can see that we have 'Other' values in gender feature. Let´s see how many of them appear within the dataset and see if we can classify them

In [ ]:
ds.gender.value_counts(normalize=False)

As we have only one individual with this classification, we can change it as we want with no important effect. In this case, it was decided to change it to Male

In [ ]:
ds.loc[ds['gender'] == 'Other', 'gender'] = 'Male'
ds.gender.value_counts(normalize=False)

## Work type assumption

Another thing that it was decided to do, is to include the 'children' work type into the 'never_worked' classification. Although this last one may refer to adults (not children) that have never worked, this is information that is also included in the 'age' feature, so combining them can be acceptable.

In [ ]:
ds.work_type.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
ds.loc[ds['work_type'] == 'children','work_type'] = 'Never_worked'

In [ ]:
ds.work_type.value_counts().plot(kind='bar', title='Count (target)');

## Unknown smoking status

About smoking status, we can see that there is a high amount of individuals as an unknown smoking status. Almost half of individuals within the dataset is classified this way, which may create some noise for evaluating the impact of this feature with the target variable.

In [ ]:
ds.smoking_status.value_counts(normalize=False)

In [ ]:
ds.smoking_status.value_counts(normalize=True).plot.pie(autopct="%.1f%%");

One thing we can do is to change some of this value based on some realistinc assumptions. In this case, it was decided to re-classify some of these individuals by age, as it is less likely for young people to smoke. First, lets check how many young people is defined as 'smokes' or 'formerly smoked'

In [ ]:
ds.loc[(ds['age'] < 18) & (ds['smoking_status'] == 'smokes'),['age','smoking_status']]

In [ ]:
ds.loc[(ds['age'] < 18) & (ds['smoking_status'] == 'formerly_smoked'),['age','smoking_status']]

We can see that there is only 9 people with less than 18 years that smoke. Although the number of people is not zero, it is a very tiny population compared to the dataset, so we decided to re-classify the people younger than 16 smoking status to 'never smoked'

In [ ]:
ds.loc[(ds['age'] < 16) & (ds['smoking_status'] == 'Unknown'),'smoking_status'] = 'never smoked'

In [ ]:
ds.smoking_status.value_counts(normalize=False)

In [ ]:
ds.smoking_status.value_counts(normalize=True).plot.pie(autopct="%.1f%%");

We can see that we have reduced unknowns from 1544 to 909, which seems to have been a good assumption. Nevertheless, the number is to high and  may affect the way this features seems to relate to stroke.

In [ ]:
ds = ds[ds['smoking_status'] != 'Unknown']

In [ ]:
ds.smoking_status.value_counts(normalize=False)

In [ ]:
ds.smoking_status.value_counts(normalize=True).plot.pie(autopct="%.1f%%");

In [ ]:
ds.shape

In [ ]:
ds

## Exploratory Data Analysis

### Count Plots for Categorical Features

In [ ]:
# Age values to ranges for plotting
ranges = [0, 18, 30, 40, 50, 60, 70, np.inf]
labels = ['0-18','18-30', '30-40', '40-50', '50-60', '60-70', '70+']

ages = pd.cut(ds['age'], bins=ranges, labels=labels)

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(15, 20))

sns.countplot(ax=axes[0,0], x= ages ,hue='stroke',data=ds).set_title('Age count plotting')
sns.countplot(ax=axes[0,1], x='gender',hue='stroke',data=ds).set_title('Gender countn plot')
sns.countplot(ax=axes[1,0], x='hypertension',hue='stroke',data=ds).set_title('Hypertension count plot')
sns.countplot(ax=axes[1,1], x='heart_disease',hue='stroke',data=ds).set_title('Heart Disease count plot')
sns.countplot(ax=axes[2,0], x='ever_married',hue='stroke',data=ds).set_title('Marriage count plot')
sns.countplot(ax=axes[2,1], x='work_type',hue='stroke',data=ds).set_title('Work Type count plot')
sns.countplot(ax=axes[3,0], x='Residence_type',hue='stroke',data=ds).set_title('Resindence Type count plot')
sns.countplot(ax=axes[3,1], x='smoking_status',hue='stroke',data=ds).set_title('Title')

Conclusions from Count Plots:
- The dataset is highly unbalanced
- Age seems to be related to stroke, as we can see that older people is more likely from suffering from it

In [ ]:
sns.countplot(x=ages ,hue='ever_married',data=ds).set_title('Age vs Ever Married')

## Unbalanced Dataset

In [ ]:
ds.stroke.value_counts(normalize=False)

In [ ]:
ds.stroke.value_counts().plot(kind='bar', title='Original Stroke Count');

In [ ]:
ds.stroke.value_counts(normalize=True).plot.pie(autopct="%.1f%%");

We can see that the target variable is Nevative (0) only for the 4.9% of the population. So, resampling the data might be necessary so that we can appreciate the relationship between the features and the target variable. There are two principal ways on doing so. Either we downsample the stroke=0 condition or we oversample the stroke=1 condition

## Random under-sampling

In [ ]:
# Class count
count_class_0, count_class_1 = ds.stroke.value_counts()

# Divide by class
ds_class_0 = ds[ds['stroke'] == 0]
ds_class_1 = ds[ds['stroke'] == 1]

In [ ]:
ds_class_0_under = ds_class_0.sample(count_class_1)
ds_under = pd.concat([ds_class_0_under, ds_class_1], axis=0)

print('Random under-sampling:')
print(ds_under.stroke.value_counts())

ds_under.stroke.value_counts().plot(kind='bar', title='Count (target)');

## Violin Plots

Now lets see what violin Plots can help us to detect with one of this balanced dataset. For this case, we are going to use the under sampled dataset

In [ ]:
ages = pd.cut(ds_under['age'], bins=ranges, labels=labels)
fig, axes = plt.subplots(4, 2, figsize=(20, 30))

sns.violinplot(ax=axes[0,0], x=ages, y='stroke', data=ds_under, title='prueba')
sns.violinplot(ax=axes[0,1], x='gender', y='stroke', data=ds_under, title='prueba')
sns.violinplot(ax=axes[1,0], x='hypertension', y='stroke', data=ds_under)
sns.violinplot(ax=axes[1,1], x='heart_disease', y='stroke', data=ds_under)
sns.violinplot(ax=axes[2,0], x='ever_married', y='stroke', data=ds_under)
sns.violinplot(ax=axes[2,1], x='work_type', y='stroke', data=ds_under)
sns.violinplot(ax=axes[3,0], x='Residence_type', y='stroke', data=ds_under)
sns.violinplot(ax=axes[3,1], x='smoking_status', y='stroke', data=ds_under)

plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15, 5))
sns.violinplot(ax=axes[0], x=ds_under["stroke"], y=ds_under["avg_glucose_level"])
sns.violinplot(ax=axes[1], x=ds_under["stroke"], y=ds_under["bmi"])
plt.show()

## Random Over-sampling

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
ds['gender'] = le.fit_transform(ds['gender'])
ds['ever_married'] = le.fit_transform(ds['ever_married'])
ds['work_type'] = le.fit_transform(ds['work_type'])
ds['Residence_type'] = le.fit_transform(ds['Residence_type'])
ds['smoking_status'] = le.fit_transform(ds['smoking_status'])

In [ ]:
print(ds['gender'].unique())
print(ds['hypertension'].unique())
print(ds['heart_disease'].unique())
print(ds['ever_married'].unique())
print(ds['work_type'].unique())
print(ds['Residence_type'].unique())
print(ds['smoking_status'].unique())
print(ds['stroke'].unique())

In [ ]:
# Class count
count_class_0, count_class_1 = ds.stroke.value_counts()

# Divide by class
ds_class_0 = ds[ds['stroke'] == 0]
ds_class_1 = ds[ds['stroke'] == 1]

In [ ]:
ds_class_1_over = ds_class_1.sample(count_class_0, replace=True)
ds_over = pd.concat([ds_class_0, ds_class_1_over], axis=0)

print('Random over-sampling:')
print(ds_over.stroke.value_counts())

ds_over.stroke.value_counts().plot(kind='bar', title='SMOTE Over-sampling');

## SMOTE Over-Sampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
labels = ds.columns[:10]

X_smote = ds[labels]
y_smote = ds['stroke']

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X_smote, y_smote)

print(y_sm.value_counts())

y_sm.value_counts().plot(kind='bar', title='Count (target)');

## Feature Selection

### Correlation

In [ ]:
df = pd.concat([X_sm, y_sm], axis=1)

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap = 'Reds')
plt.show()

In [ ]:
cor = df.corr()
cor_target = abs(cor['stroke'])
relevant_features = cor_target[cor_target > 0.5]
relevant_features

By using this method for feature selection, we can observe that there is only ine feature considered to be highly corrated. But let´s now use another method called Backward Elimination to see if it tell us something different.

### Backward Elimation - SMOTE

In [ ]:
import statsmodels.api as sm

In [ ]:
X = sm.add_constant(X_sm)

In [ ]:
model = sm.OLS(y_sm,X).fit()
model.pvalues

In [ ]:
cols = list(X_sm.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X = X_sm[cols]
    X = sm.add_constant(X)
    model = sm.OLS(y_sm,X).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
selected_features_BE.remove('ever_married')

In [ ]:
X_sm = X_sm[cols]
X_sm.columns

# 3. Data preparation

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
import os
import warnings
#from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category = ConvergenceWarning)

In [ ]:
normalization = lambda x:(x-x.min()) / (x.max()-x.min())
transformColumns2 = X_sm
X_sm = normalization(transformColumns2)

In [ ]:
from sklearn.model_selection import train_test_split

### Splitting into train and test

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_sm,y_sm,test_size=0.25, random_state=0)
pd.DataFrame({"Train Row Count":[x_train.shape[0],y_train.shape[0]],
              "Test Row Count":[x_test.shape[0],y_test.shape[0]]},
             index=["X (Predictors)","Y (Target)"])

In [ ]:
y_train = y_train.values.ravel()

# 4. Modeling - SMOTE Oversampling

In [ ]:
# Decision Tree
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)
y_pred_dtc = dtc.predict(x_test)
dtc_acc = accuracy_score(y_test,y_pred_dtc)

In [ ]:
# Logistic Regression
logr = LogisticRegression()
logr.fit(x_train,y_train)
y_pred_logr = logr.predict(x_test)
logr_acc = accuracy_score(y_test,y_pred_logr)

In [ ]:
# Naive Bayes
gnb = GaussianNB()
gnb.fit(x_train,y_train)
y_pred_gnb = gnb.predict(x_test)
gnb_acc = accuracy_score(y_test,y_pred_gnb)

In [ ]:
# K Neighbors Classifier
knn = KNeighborsClassifier( metric='minkowski')
knn.fit(x_train,y_train)
y_pred_knn = knn.predict(x_test)
knn_acc = accuracy_score(y_test,y_pred_knn)

In [ ]:
# Random Forest
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
y_pred_rfc = rfc.predict(x_test)
rfc_acc = accuracy_score(y_test,y_pred_rfc)

In [ ]:
# Support Vector Machine
svm = SVC()
svm.fit(x_train,y_train)
y_pred_svm = svm.predict(x_test)
svm_acc = accuracy_score(y_test,y_pred_svm)

In [ ]:
# Neural Network
nnc = MLPClassifier()
nnc.fit(x_train,y_train)
y_pred_nnc = nnc.predict(x_test)
nnc_acc = accuracy_score(y_test,y_pred_nnc)

In [ ]:
# Xgboost Classifier
xgboast = xgb.XGBClassifier()
xgboast.fit(x_train, y_train)
xgboast = xgboast.score(x_test,y_test)

# 5. Evaluation

In [ ]:
pd.DataFrame({"Algorithms":["Decision Tree","Logistic Regression","Naive Bayes","K Neighbors Classifier","Random Ferest","Neural Network","SVM","Xgboost Classifier"],
              "Scores":[dtc_acc,logr_acc,gnb_acc,knn_acc,rfc_acc,nnc_acc,svm_acc,xgboast]})

In [ ]:
# Cross validation test
models = []
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('K Neighbors Classifier', KNeighborsClassifier()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Neural Network', MLPClassifier()))
models.append(('SVM', SVC()))
models.append(('Xgboost', XGBClassifier()))

In [ ]:
# evaluate each model in turning kfold results
results_boxplot = []
names = []
results_mean = []
results_std = []
p,t = X_sm.values, y_sm.values.ravel()
for name, model in models:
    cv_results = cross_val_score(model, p,t, cv=10)
    results_boxplot.append(cv_results)
    results_mean.append(cv_results.mean())
    results_std.append(cv_results.std())
    names.append(name)
pd.DataFrame({"Algorithm":names,
                                "Accuracy Mean":results_mean,
                                "Accuracy":results_std})

In [ ]:
# boxplot algorithm comparison

fig = plt.figure(figsize=(7,5))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results_boxplot)
ax.set_xticklabels(names)
plt.xticks(rotation=45)

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,roc_auc_score,roc_curve

In [ ]:
rfc = RandomForestClassifier()
model_name = 'RFC'
rfc.fit(x_train,y_train)
y_pred_rfc = rfc.predict(x_test)

rfc_acc = accuracy_score(y_test,y_pred_rfc)
f1 = f1_score(y_test,y_pred_rfc)
rfc_auc = roc_auc_score(y_test,y_pred_rfc)


auc_dtc = roc_auc_score(y_test,y_pred_dtc)


In [ ]:
print(f'{model_name.upper()} Validation Score:\nAccuracy Score:{rfc_acc:.3f} F1 Score:{f1:.3f}')
mat = confusion_matrix(y_test,y_pred_rfc)
sns.heatmap(mat,cmap='Blues',annot=True,fmt='g')
plt.show()

In [ ]:
r_probs = [0 for _ in range(len(y_test))]

rf_probs = rfc.predict_proba(x_test)
rf_probs = rf_probs[:, 1]

dtc_probs = dtc.predict_proba(x_test)
dtc_probs = dtc_probs[:, 1]

In [ ]:
r_auc = roc_auc_score(y_test, r_probs)
rf_auc = roc_auc_score(y_test, rf_probs)

dtc_auc = roc_auc_score(y_test, dtc_probs)

In [ ]:
r_fpr, r_tpr, _ = roc_curve(y_test, r_probs)
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probs)

dtc_fpr, dtc_tpr, _ = roc_curve(y_test, dtc_probs)

In [ ]:
print('Random (chance) Prediction: AUC = %.3f' % (r_auc))
print('Random Forest: AUC = %.3f' % (rf_auc))

plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUC = %0.3f)' % r_auc)
plt.plot(rf_fpr, rf_tpr, marker='.', label='Random Forest (AUC = %0.3f)' % rf_auc)
#plt.plot(dtc_fpr, dtc_tpr, marker='.', label='Decision Tree (AUC = %0.3f)' % dtc_auc)

# Title
plt.title('ROC Plot')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Show legend
plt.legend() # 
# Show plot
plt.show()